# Preference Alignment with Direct Preference Optimization (DPO)

This notebook will guide you through the process of fine-tuning a language model using Direct Preference Optimization (DPO). We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with DPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p> 
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Select a dataset that relates to a real-world use case you’re interested in, or use the model you trained in 
        <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
import yaml

# Load the config file
with open("../../config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Extract the token
hf_token = config.get("huggingface_hub", {}).get("token")

if hf_token:
    # Log in to Hugging Face Hub
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Access token not found in config.yaml!")
#login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Successfully logged in to Hugging Face!


## Import libraries


In [9]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig


## Format dataset

In [10]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
dataset = load_dataset(path="argilla/ultrafeedback-binarized-preferences", split="train")

In [11]:
# Inspect the dataset structure
print("Dataset Structure:")
print(dataset)

Dataset Structure:
Dataset({
    features: ['source', 'instruction', 'chosen_response', 'rejected_response', 'chosen_avg_rating', 'rejected_avg_rating', 'chosen_model'],
    num_rows: 63619
})


In [13]:
# TODO: 🐕 If your dataset is not represented as conversation lists, you can use the `process_dataset` function to convert it.
# Preprocess dataset to extract conversations
from datasets import Dataset  # Import the Dataset class

from datasets import Dataset

def preprocess_dataset_for_dpo(dataset):
    """
    Preprocess the dataset to add explicit prompts for chosen and rejected responses,
    including a required 'prompt' field for DPOTrainer.

    Args:
        dataset (Dataset): Input dataset with fields:
            ['source', 'instruction', 'chosen_response', 'rejected_response', 
             'chosen_avg_rating', 'rejected_avg_rating', 'chosen_model'].

    Returns:
        Dataset: Updated dataset with explicit prompt extraction.
    """
    def format_conversation(entry):
        """
        Formats each dataset entry into the required structure for DPO training.
        """
        prompt = f"System: You are a helpful AI assistant named SmolLM, trained by Hugging Face.\nUser: {entry['instruction']}"
        chosen_response = entry['chosen_response']
        rejected_response = entry['rejected_response']

        return {
            "prompt": prompt,
            "chosen": f"{prompt}\nAssistant: {chosen_response}",
            "rejected": f"{prompt}\nAssistant: {rejected_response}",
            "ratings": {
                "chosen_avg_rating": entry["chosen_avg_rating"],
                "rejected_avg_rating": entry["rejected_avg_rating"]
            }
        }

    # Apply the formatting logic to the dataset
    processed_dataset = dataset.map(format_conversation, batched=False)
    return processed_dataset

# Preprocess the dataset for DPO training
processed_data = preprocess_dataset_for_dpo(dataset)

# Verify the processed data
print(processed_data[0])

{'source': 'evol_instruct', 'instruction': 'Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here\'s some starter code to help you out:\n#include <iostream>\n#include <string>\nusing namespace std;\nint main() {\n    string country;\n    // prompt user for input\n    cout << "Enter the name of a country: ";\n    cin >> country;\n    // check if country borders the Mediterranean Sea\n    // [C++ code]\n    return 0;\n}', 'chosen_response': 'Here\'s a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea:\n\n#include <iostream>\n#include <string>\n#include <set>\n#include <map>\n#include <algorithm>\n\nusing namespace std;\n\nint main() {\n    // store countries and their bordering seas in a map\n    map<string, set<string>> countries;\n    countries["Algeria"] = {"Mediterranean Sea", "North African Coast"};\n    countries["France"] = {"Mediterranean S

## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 change the model to the path or repo id of the model you trained in <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>


In [14]:
# TODO: 🦁 change the model to the path or repo id of the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb)

#model_name = "/Users/ohnmarhtun/lab/smol-course/1_instruction_tuning/notebooks/SmolLM2-135M-FT2"
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO-argilla1"
finetune_tags = ["smol-course", "module_1"]



## Train model with DPO

In [15]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=200,
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output_argilla1",
    # Number of steps for learning rate warmup
    warmup_steps=100,
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to=None,
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    use_mps_device=device == "mps",
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
)

/Users/ohnmarhtun/anaconda3/envs/smol/lib/python3.11/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [16]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=processed_data,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    beta=0.1,
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    max_length=1536,
)

/Users/ohnmarhtun/anaconda3/envs/smol/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/ohnmarhtun/anaconda3/envs/smol/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/Users/ohnmarhtun/anaconda3/envs/smol/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:475: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(


Applying chat template to train dataset:   0%|          | 0/63619 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/63619 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2072 > 2048). Running this sequence through the model will result in indexing errors
max_steps is given, it will override any value given in num_train_epochs


In [17]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

# Save to the huggingface hub if login (HF_TOKEN is set)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.692900
4,0.692600
5,0.692800
6,0.692700
7,0.691100
8,0.684400
9,0.687700
10,0.693100


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `DPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.